# 4.8 Grouping Data & Aggregating Variables

## Content List
> * Import Libraries
> * Step Two- Find mean for order numbers
> * Step three- Difference in subsets
> * Step four- Create flag based on loyalty customers
> * Step five- Aggregations 
> * Step six- Create spending flag
> * Step seven- Create frequency flag
> * Step eight- export

## Import Libraries and Data

In [1]:
#import libraries
import pandas as pd
import numpy as np
import os

In [2]:
#folder path
path = r'C:\Users\kaydo\OneDrive\Desktop\Python\Data\Prepared Data'

In [3]:
# import orders_products_merged.pkl
ords_prods_merged = pd.read_pickle(r'C:\Users\kaydo\OneDrive\Desktop\Python\Data\Prepared Data\orders_products_merged_2.pkl')

# Step Two

In [4]:
#Group by department and find mean for order numbers
ords_prods_merged.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


# Step Three

More departments are included with this versus analyzing the subset. More of the department means were higher with the subset as well.

# Step Four

In [5]:
#Generate max orders 
ords_prods_merged['max_order'] = ords_prods_merged.groupby(['user_id'])['order_number'].transform(np.max)

In [6]:
#Check for new column
ords_prods_merged.head()

,order_id,user_id,eval_set,order_number,Order_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge,price_range_loc,busiest_day,Busiest_days,busiest_period_of_day,max_order
0,2539329,1,prior,1,2,8,NaN,196,1,0,Soda,77,7,9.0,both,Mid range product,Regularly busy,Regularly busy days,Average orders,10
1,2398795,1,prior,2,3,7,15.0,196,1,1,Soda,77,7,9.0,both,Mid range product,Regularly busy,Slowest days,Average orders,10
2,473747,1,prior,3,3,12,21.0,196,1,1,Soda,77,7,9.0,both,Mid range product,Regularly busy,Slowest days,Most orders,10
3,2254736,1,prior,4,4,7,29.0,196,1,1,Soda,77,7,9.0,both,Mid range product,Least busy,Slowest days,Average orders,10
4,431534,1,prior,5,4,15,28.0,196,1,1,Soda,77,7,9.0,both,Mid range product,Least busy,Slowest days,Most orders,10


In [7]:
#Create flag
ords_prods_merged.loc[ords_prods_merged['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'
ords_prods_merged.loc[(ords_prods_merged['max_order'] <= 40) & (ords_prods_merged['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'
ords_prods_merged.loc[ords_prods_merged['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [8]:
# Check output
ords_prods_merged['loyalty_flag'].value_counts(dropna = False)

Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: loyalty_flag, dtype: int64

# Step Five

In [9]:
# Perform multiple aggregations
ords_prods_merged.groupby('loyalty_flag').agg({'prices':['mean', 'min', 'max', 'sum']})

prices                         
                      mean  min   max           sum
loyalty_flag                                       
Loyal customer    7.772831  1.0  25.0  7.993651e+07
New customer      7.800320  1.0  25.0  4.870512e+07
Regular customer  7.797431  1.0  25.0  1.237981e+08

# Step Six

In [10]:
#Group by user id and generate column for avg prices
ords_prods_merged['avg_prices'] = ords_prods_merged.groupby(['user_id'])['prices'].transform(np.mean)

In [11]:
#Check for new column
ords_prods_merged.head()

,order_id,user_id,eval_set,order_number,Order_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,department_id,prices,_merge,price_range_loc,busiest_day,Busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_prices
0,2539329,1,prior,1,2,8,NaN,196,1,0,...,7,9.0,both,Mid range product,Regularly busy,Regularly busy days,Average orders,10,New customer,6.367797
1,2398795,1,prior,2,3,7,15.0,196,1,1,...,7,9.0,both,Mid range product,Regularly busy,Slowest days,Average orders,10,New customer,6.367797
2,473747,1,prior,3,3,12,21.0,196,1,1,...,7,9.0,both,Mid range product,Regularly busy,Slowest days,Most orders,10,New customer,6.367797
3,2254736,1,prior,4,4,7,29.0,196,1,1,...,7,9.0,both,Mid range product,Least busy,Slowest days,Average orders,10,New customer,6.367797
4,431534,1,prior,5,4,15,28.0,196,1,1,...,7,9.0,both,Mid range product,Least busy,Slowest days,Most orders,10,New customer,6.367797


In [13]:
#Create a spending flag for each user based on the average price across all their orders using the following criteria: If the mean of the prices of products purchased by a user is lower than 10, then flag them as a “Low spender.”If the mean of the prices of products purchased by a user is higher than or equal to 10, then flag them as a “High spender.”
ords_prods_merged.loc[ords_prods_merged['avg_prices'] <10, 'spending_flag']= 'Low Spender'
ords_prods_merged.loc[ords_prods_merged['avg_prices'] >= 10, 'spending_flag']= 'High Spender'

In [14]:
#check output
ords_prods_merged.head(15)

,order_id,user_id,eval_set,order_number,Order_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,prices,_merge,price_range_loc,busiest_day,Busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_prices,spending_flag
0,2539329,1,prior,1,2,8,NaN,196,1,0,...,9.0,both,Mid range product,Regularly busy,Regularly busy days,Average orders,10,New customer,6.367797,Low Spender
1,2398795,1,prior,2,3,7,15.0,196,1,1,...,9.0,both,Mid range product,Regularly busy,Slowest days,Average orders,10,New customer,6.367797,Low Spender
2,473747,1,prior,3,3,12,21.0,196,1,1,...,9.0,both,Mid range product,Regularly busy,Slowest days,Most orders,10,New customer,6.367797,Low Spender
3,2254736,1,prior,4,4,7,29.0,196,1,1,...,9.0,both,Mid range product,Least busy,Slowest days,Average orders,10,New customer,6.367797,Low Spender
4,431534,1,prior,5,4,15,28.0,196,1,1,...,9.0,both,Mid range product,Least busy,Slowest days,Most orders,10,New customer,6.367797,Low Spender
5,3367565,1,prior,6,2,7,19.0,196,1,1,...,9.0,both,Mid range product,Regularly busy,Regularly busy days,Average orders,10,New customer,6.367797,Low Spender
6,550135,1,prior,7,1,9,20.0,196,1,1,...,9.0,both,Mid range product,Regularly busy,Busiest days,Most orders,10,New customer,6.367797,Low Spender
7,3108588,1,prior,8,1,14,14.0,196,2,1,...,9.0,both,Mid range product,Regularly busy,Busiest days,Most orders,10,New customer,6.367797,Low Spender
8,2295261,1,prior,9,1,16,0.0,196,4,1,...,9.0,both,Mid range product,Regularly busy,Busiest days,Most orders,10,New customer,6.367797,Low Spender
9,2550362,1,prior,10,4,8,30.0,196,1,1,...,9.0,both,Mid range product,Least busy,Slowest days,Average orders,10,New customer,6.367797,Low Spender


In [15]:
#check counts of spending flag values
ords_prods_merged['spending_flag'].value_counts(dropna = False)

Low Spender     32285165
High Spender      119694
Name: spending_flag, dtype: int64

# Step Seven

In order to send relevant notifications to users within the app (for instance, asking users if they want to buy the same item again), the Instacart team wants you to determine frequent versus non-frequent customers. Create an order frequency flag that marks the regularity of a user’s ordering behavior according to the median in the “days_since_prior_order” column. The criteria for the flag should be as follows:
If the median of “days_since_prior_order” is higher than 20, then the customer should be labeled a “Non-frequent customer.”
If the median is higher than 10 and lower than or equal to 20, then the customer should be labeled a “Regular customer.”
If the median is lower than or equal to 10, then the customer should be labeled a “Frequent customer.”

In [17]:
#Group by user id and generate column for avg prices
ords_prods_merged['median_freq'] = ords_prods_merged.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [22]:
#Create a flag for frequent/ non-frequent customers
ords_prods_merged.loc[ords_prods_merged['median_freq'] > 20, 'order_freq_flag']= 'Non-Frequent Customer'
ords_prods_merged.loc[ords_prods_merged['median_freq'] <= 10, 'order_freq_flag']= 'Frequent Customer'
ords_prods_merged.loc[(ords_prods_merged['median_freq'] > 10) & (ords_prods_merged['median_freq'] <= 20), 'order_freq_flag']= 'Regular Customer'

In [23]:
#check output
ords_prods_merged.head(15)

,order_id,user_id,eval_set,order_number,Order_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,price_range_loc,busiest_day,Busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_prices,spending_flag,median_freq,order_freq_flag
0,2539329,1,prior,1,2,8,NaN,196,1,0,...,Mid range product,Regularly busy,Regularly busy days,Average orders,10,New customer,6.367797,Low Spender,20.5,Non-Frequent Customer
1,2398795,1,prior,2,3,7,15.0,196,1,1,...,Mid range product,Regularly busy,Slowest days,Average orders,10,New customer,6.367797,Low Spender,20.5,Non-Frequent Customer
2,473747,1,prior,3,3,12,21.0,196,1,1,...,Mid range product,Regularly busy,Slowest days,Most orders,10,New customer,6.367797,Low Spender,20.5,Non-Frequent Customer
3,2254736,1,prior,4,4,7,29.0,196,1,1,...,Mid range product,Least busy,Slowest days,Average orders,10,New customer,6.367797,Low Spender,20.5,Non-Frequent Customer
4,431534,1,prior,5,4,15,28.0,196,1,1,...,Mid range product,Least busy,Slowest days,Most orders,10,New customer,6.367797,Low Spender,20.5,Non-Frequent Customer
5,3367565,1,prior,6,2,7,19.0,196,1,1,...,Mid range product,Regularly busy,Regularly busy days,Average orders,10,New customer,6.367797,Low Spender,20.5,Non-Frequent Customer
6,550135,1,prior,7,1,9,20.0,196,1,1,...,Mid range product,Regularly busy,Busiest days,Most orders,10,New customer,6.367797,Low Spender,20.5,Non-Frequent Customer
7,3108588,1,prior,8,1,14,14.0,196,2,1,...,Mid range product,Regularly busy,Busiest days,Most orders,10,New customer,6.367797,Low Spender,20.5,Non-Frequent Customer
8,2295261,1,prior,9,1,16,0.0,196,4,1,...,Mid range product,Regularly busy,Busiest days,Most orders,10,New customer,6.367797,Low Spender,20.5,Non-Frequent Customer
9,2550362,1,prior,10,4,8,30.0,196,1,1,...,Mid range product,Least busy,Slowest days,Average orders,10,New customer,6.367797,Low Spender,20.5,Non-Frequent Customer


In [24]:
#check counts of Frequent Customers
ords_prods_merged['order_freq_flag'].value_counts(dropna = False)

Frequent Customer        21559853
Regular Customer          7208564
Non-Frequent Customer     3636437
NaN                             5
Name: order_freq_flag, dtype: int64

# Step Nine

In [25]:
ords_prods_merged.to_pickle(os.path.join(path, 'orders_products_merged_aggregated.pkl'))